In [11]:
# system imports
import os
import sys

# data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import seaborn as sns

# signal processing
from scipy import signal
from scipy.ndimage import label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz


# misc
import warnings

import glob

##
import pytz
import datetime as dt
import math
import seaborn as sns

## Psychopy Related Functions

In [12]:
def read_manipulate_psychopy(psychopy_path):
    
    psychopy_df = pd.read_csv(psychopy_path)
    col_list = [col for col in psychopy_df.columns if col.endswith('_ts')]
    col_list.insert(0, "Reference_time")
    psychopy_df_selected = psychopy_df[col_list]
    
    return psychopy_df_selected


def get_nonNan_list_psychopy(psychopy_df, col_name):
    
    
    selected_vals=[val for val in psychopy_df[col_name].to_list() if not(math.isnan(val))]
    
    return selected_vals



#######---------------------------------------------------
###### ------------------Shimmer--------------------------
###----------------------------------------------------------


def read_shimmer_sensor(sensor_file_path):
    
    shimmer_df = pd.read_csv(sensor_file_path, sep='\t', low_memory=False)
    shimmer_df = shimmer_df.reset_index()
    shimmer_df.columns = shimmer_df.iloc[0]
    shimmer_df.drop([0, 1], axis=0, inplace=True)
    shimmer_df=shimmer_df.reset_index(drop=True)
    
    return shimmer_df

def standardize_timestamps_shimmer(shimmer_df, timestamps_col_name):
    
    timesstamps_list = shimmer_df[timestamps_col_name].to_list()
    new_timestamps_list = [float(val)/1000 for val in  timesstamps_list]
    
    shimmer_df[timestamps_col_name] = new_timestamps_list
    
    return shimmer_df



def get_offset_timestamp(timestamp, offset_mins):
    
    time_zone = 'Europe/Berlin'
    tz = pytz.timezone(time_zone)
    local_time = dt.datetime.fromtimestamp(timestamp, tz)
    time_change = dt.timedelta(minutes=offset_mins)
    new_time = local_time + time_change
    new_timestamp =  dt.datetime.timestamp(new_time)
    return new_timestamp

def get_list_timestamp_interest(starting_timestamp, list_offset_mins):
    
    starting_timestamp_list=[]
    
    for offset_min in list_offset_mins:
        starting_timestamp_list.append(starting_timestamp)
        timestamp_offset = get_offset_timestamp(starting_timestamp, offset_min)
        starting_timestamp = timestamp_offset  
     
    
    #starting_timestamp_list = sorted(starting_timestamp_list, key = lambda x:float(x))
    return starting_timestamp_list
        
        

def slice_df_wrt_timestamps(df, start_timestamp, end_timestamp, timestamps_col):
    
    sliced_df=df[(df[timestamps_col]>= start_timestamp) & (df[timestamps_col] <= end_timestamp)]
    
    return sliced_df


def from_str_to_float(str_list):
    
    float_array =[float(val) for val in str_list]
    
    return float_array

def col_from_str_float (df, col_name):
    
    str_list = df[col_name].values
    
    float_array =[float(val) for val in str_list]
    
    df[col_name] = float_array
    
    return df

## HRV: Time domain analysis

In [34]:
def detect_peaks(ecg_signal, threshold=0.3, qrs_filter=None):
    '''
    Peak detection algorithm using cross corrrelation and threshold 
    '''
    if qrs_filter is None:
        # create default qrs filter, which is just a part of the sine function
        t = np.linspace(1.5 * np.pi, 3.5 * np.pi, 15)
        qrs_filter = np.sin(t)
    
    # normalize data
    ecg_signal = (ecg_signal - ecg_signal.mean()) / ecg_signal.std()

    # calculate cross correlation
    similarity = np.correlate(ecg_signal, qrs_filter, mode="same")
    similarity = similarity / np.max(similarity)

    # return peaks (values in ms) using threshold
    return ecg_signal[similarity > threshold].index, similarity



def group_peaks(p, threshold=5):
    '''
    The peak detection algorithm finds multiple peaks for each QRS complex. 
    Here we group collections of peaks that are very near (within threshold) and we take the median index 
    '''
    # initialize output
    output = np.empty(0)

    # label groups of sample that belong to the same peak
    peak_groups, num_groups = label(np.diff(p) < threshold)

    # iterate through groups and take the mean as peak index
    for i in np.unique(peak_groups)[1:]:
        peak_group = p[np.where(peak_groups == i)]
        output = np.append(output, np.median(peak_group))
    return output

def group_peaks_from_ind_to_msec(grouped_peaks_ind, sampling_freq):
    
    
    #grouped_peak_ascending = sorted(grouped_peaks_ind, key = lambda x:float(x))
    
    #grouped_peak_ascending_np=np.array(grouped_peak_ascending)
    grouped_peak_sec = grouped_peaks_ind*(1/sampling_freq)
    
    grouped_peak_msec = grouped_peak_sec*1000
    
    return grouped_peak_msec
    


def timedomain(rr):
    results = {}

    hr = 60000/rr
    
    results['Mean RR (ms)'] = np.mean(rr)
    results['STD RR/SDNN (ms)'] = np.std(rr)
    #results['Mean HR (Kubios\' style) (beats/min)'] = 60000/np.mean(rr)
    #results['Mean HR (beats/min)'] = np.mean(hr)
    #results['STD HR (beats/min)'] = np.std(hr)
    #results['Min HR (beats/min)'] = np.min(hr)
    #results['Max HR (beats/min)'] = np.max(hr)
    results['RMSSD (ms)'] = np.sqrt(np.mean(np.square(np.diff(rr))))
    results['NNxx'] = np.sum(np.abs(np.diff(rr)) > 100)*1
    results['pNNxx (%)'] = 100 * np.sum((np.abs(np.diff(rr)) > 100)*1) / len(rr)
    return results



def filter_rr(rr, throshold_high=0.5, threshold_low=0.5):
    
    #outlier_high = np.mean(rr) + throshold_high * np.std(rr)
    outlier_high =1500
    #print(outlier_high)
    
    rr_clean_high = [rr_val for rr_val in rr if rr_val <  outlier_high]
    
    rr_clean_high = np.array(rr_clean_high)
    
    #outlier_low = np.mean(rr_clean_high) - threshold_low*np.std(rr_clean_high)
    outlier_low = 400
    #print(outlier_low)
    
    rr_clean_high_low = [rr_val for rr_val in rr_clean_high if rr_val >  outlier_low]
    
    rr_clean_high_low= np.array(rr_clean_high_low)
    
    return rr_clean_high_low




def min_max_norm(all_rr_one_list, all_rr_sublist):
    
    mean_norm_list =[]
    std_norm_list = []
    
    all_rr_one_list = np.array(all_rr_one_list)
    max_val = np.max(all_rr_one_list)
    min_val =np.min(all_rr_one_list)
    
    for sel_list in all_rr_sublist:
        
        sel_list_np = np.array(sel_list)
        
        range_list = max_val-min_val
        
        sel_lis_np_sub = sel_list_np - min_val
        
        sel_lis_norm= sel_lis_np_sub/range_list
        
        mean_norm = np.mean(sel_lis_norm)
        mean_norm_list.append(mean_norm)
        
        std_norm  = np.std(sel_lis_norm)   
        std_norm_list.append(std_norm)
        #print(mean_norm)
        #print(std_norm)
                
        
    return mean_norm_list, std_norm_list
        
    


def get_plot_ranges(start=10, end=20, n=5):
    '''
    Make an iterator that divides into n or n+1 ranges. 
    - if end-start is divisible by steps, return n ranges
    - if end-start is not divisible by steps, return n+1 ranges, where the last range is smaller and ends at n
    
    # Example:
    >> list(get_plot_ranges())
    >> [(0.0, 3.0), (3.0, 6.0), (6.0, 9.0)]

    '''
    distance = end - start
    for i in np.arange(start, end, np.floor(distance/n)):
        yield (int(i), int(np.minimum(end, np.floor(distance/n) + i)))

## Path and files

In [35]:
data_dir_path = "/home/muhammad/Desktop/Datasets/data_sony_digiRelax/wp3"
#data_dir_path = "D:/Datasets/data_sony_digiRelax/study"
sub_dirs=next(os.walk(data_dir_path))[1]

In [36]:
sub_dirs

['VP020_221123',
 'VP009_101123',
 'VP019_201123',
 'VP014_151123',
 'VP006_091123',
 'VP013_151123',
 'VP018_201123',
 'VP021_221123',
 'VP010_131123',
 'VP007_091123',
 'VP015_161123',
 'VP011_131123',
 'VP016_161123',
 'VP004_081123',
 'VP012_141123',
 'VP022_231123',
 'VP005_081123',
 'VP008_101123',
 'VP017_171123']

In [37]:
def get_mean_std_event_list(psychopy_df, shimmer_df_selected, event_interest_list_new, offset_min_list_new, timestamp_sensor_col_name, data_col_name):

    mean_cumm_list = []
    std_cumm_list  = []
    event_interets_plot = []
    
    all_rr_one_list = []
    all_rr_sublist = []
    
    
    
    
    


    ##---outer loop to hnadle muultiple offsets from the selected events 
    for ind, event_selected in enumerate(event_interest_list_new):

        offset_list_selected_event = offset_min_list_new[ind]

        #print(event_selected)

        current_event_timestamp = get_nonNan_list_psychopy(psychopy_df, event_selected)[0]

        offset_min_list_selected = offset_min_list_new[ind]
        
        acc_min_print = 0

        for offset_min in offset_min_list_selected:
            
            offset_min_print = offset_min + acc_min_print
            
            acc_min_print = offset_min_print

            event_plot = event_selected +" "+ str(offset_min_print) + " min"
            
            print(event_plot)
            
            event_interets_plot.append(event_plot)

            timestamp_offset = get_offset_timestamp(current_event_timestamp, offset_min)

            start_end_time_list = [current_event_timestamp, timestamp_offset]

            timestamp_start_slice = min(start_end_time_list)

            timestamp_end_slice = max(start_end_time_list)

            sensor_df_ii=slice_df_wrt_timestamps(shimmer_df_selected, timestamp_start_slice, timestamp_end_slice, timestamp_sensor_col_name)
            
            #print(sensor_df_ii)
            
            ecg_df_ii=sensor_df_ii.reset_index(drop=True)
            
            
            #cond_slice_main = (ecg_df_ii[timestamp_sensor_col_name] >= timestamp_start_slice) & (ecg_df_ii[timestamp_sensor_col_name] < timestamp_end_slice)
            
            ecg_slice = ecg_df_ii.heartrate
            
            peaks, similarity = detect_peaks(ecg_slice, threshold=0.3)
            grouped_peaks = group_peaks(peaks)
            grouped_peak_msec =group_peaks_from_ind_to_msec(grouped_peaks, 256)
            rr = np.diff(grouped_peak_msec)
            #print(timedomain(rr))
            cleaned_rr = filter_rr(rr)
            mean_rr = np.mean(cleaned_rr) 
            std_rr = np.std(cleaned_rr)
            
            mean_cumm_list.append(mean_rr)
            std_cumm_list.append(std_rr)
            
            all_rr_one_list.extend(cleaned_rr)
            all_rr_sublist.append(cleaned_rr)
            
            
            #event_interets_plot
            
            #print(np.mean(cleaned_rr))
            #print(np.std(cleaned_rr))
            


            current_event_timestamp = timestamp_offset
            
            
            
    return mean_cumm_list, std_cumm_list, event_interets_plot, all_rr_one_list, all_rr_sublist
        

## Fetching and slicing information

In [38]:
event_interest_list_new = ["baseline_instruction_ts", "tsst_prep_ts", "tsst_pres_ts", "relaxation_prep_ts", "saliva_probe_4_ts", "saliva_probe_4_ts"]
offset_min_list_new  = [[3], [5], [5,5], [5, 5, 5], [-5], [5]]


In [39]:
dest_path ="/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp3/results_22sub"
#dest_path = "D:/Datasets/analysis_digirelax"
designated_folder = "hrv_analysis"
comp_dest_path = os.path.join(dest_path, designated_folder)

isexist = os.path.exists(comp_dest_path)

if not isexist:
    os.makedirs(comp_dest_path)
    print("The new directory is created")

In [40]:


for sub_dir in sub_dirs:
    
    sub_dir_path= os.path.join(data_dir_path, sub_dir)
    psychopy_file_path = os.path.join(sub_dir_path, "Psychopy_data")
    psychopy_file_path = glob.glob(psychopy_file_path+ "/*.csv")[0]
    #print(psychopy_file_path)
    psychopy_df = read_manipulate_psychopy(psychopy_file_path)
    
    shimmer_files_subdir = os.path.join(sub_dir_path, "Shimmer_data")
    
    shimmer_files_subdir_subdir_name = next(os.walk(shimmer_files_subdir))[1][0]
    
    shimmer_files_parent_path = os.path.join(shimmer_files_subdir, shimmer_files_subdir_subdir_name)
    
    shimmer_files_name = glob.glob(shimmer_files_parent_path+ "/*.csv")
    
    for sh_file in shimmer_files_name:
        
        if '_6B1E_' in sh_file:
            hr_shimmer_path = sh_file
            
    fol_path, file_name=os.path.split(psychopy_file_path)
    sub_id=file_name[:-4]
    print(sub_id)
    
    ## heart rate
    timestamp_hr_col = "Shimmer_6B1E_Timestamp_Unix_CAL"
    ecg_col_name = "Shimmer_6B1E_ECG_LL-LA_24BIT_CAL"
    
    hr_df=read_shimmer_sensor(hr_shimmer_path)
    hr_df = standardize_timestamps_shimmer(hr_df, timestamp_hr_col)
    
    selected_ecg_df = hr_df[[timestamp_hr_col, ecg_col_name]]
    selected_ecg_df = selected_ecg_df.copy()
    selected_ecg_df['heartrate'] = selected_ecg_df[ecg_col_name]
    selected_ecg_df=col_from_str_float(selected_ecg_df, "heartrate")
    
    mean_hrv_list,std_hrv_list, event_interets_plot, all_rr_one_list, all_rr_sublist =  get_mean_std_event_list(psychopy_df, selected_ecg_df, event_interest_list_new, offset_min_list_new, timestamp_hr_col, "heartrate")
    
    norm_mean_hrv, norm_std_hrv = min_max_norm(all_rr_one_list, all_rr_sublist)
    
    export_dict= {"subject_id":sub_id , 'event_interst': event_interets_plot, "mean_hrv":mean_hrv_list , "std_hrv": std_hrv_list , "norm_mean_hrv":norm_mean_hrv,  "norm_std_hrv": norm_std_hrv}
    
     
    dest_path_csv = os.path.join(comp_dest_path,file_name)
    df_export = pd.DataFrame.from_dict(export_dict)
    df_export.to_csv(dest_path_csv)
    
    
    
    
    
    
    

VP020_221123_DigiRelax_Experiment_2023-11-22_09h48.04.766
baseline_instruction_ts 3 min
tsst_prep_ts 5 min
tsst_pres_ts 5 min
tsst_pres_ts 10 min
relaxation_prep_ts 5 min
relaxation_prep_ts 10 min
relaxation_prep_ts 15 min
saliva_probe_4_ts -5 min
saliva_probe_4_ts 5 min
VP009_101123_DigiRelax_Experiment_2023-11-10_14h16.47.223
baseline_instruction_ts 3 min
tsst_prep_ts 5 min
tsst_pres_ts 5 min
tsst_pres_ts 10 min
relaxation_prep_ts 5 min
relaxation_prep_ts 10 min
relaxation_prep_ts 15 min
saliva_probe_4_ts -5 min
saliva_probe_4_ts 5 min
VP019_201123_DigiRelax_Experiment_2023-11-20_14h48.37.367
baseline_instruction_ts 3 min
tsst_prep_ts 5 min
tsst_pres_ts 5 min
tsst_pres_ts 10 min
relaxation_prep_ts 5 min
relaxation_prep_ts 10 min
relaxation_prep_ts 15 min
saliva_probe_4_ts -5 min
saliva_probe_4_ts 5 min
VP014_151123_DigiRelax_Experiment_2023-11-15_14h23.06.117
baseline_instruction_ts 3 min
tsst_prep_ts 5 min
tsst_pres_ts 5 min
tsst_pres_ts 10 min
relaxation_prep_ts 5 min
relaxation_pr

/home/muhammad/miniconda3/envs/sql_dep/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/muhammad/miniconda3/envs/sql_dep/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/muhammad/miniconda3/envs/sql_dep/lib/python3.9/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/muhammad/miniconda3/envs/sql_dep/lib/python3.9/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/muhammad/miniconda3/envs/sql_dep/lib/python3.9/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


baseline_instruction_ts 3 min
tsst_prep_ts 5 min
tsst_pres_ts 5 min
tsst_pres_ts 10 min
relaxation_prep_ts 5 min
relaxation_prep_ts 10 min
relaxation_prep_ts 15 min
saliva_probe_4_ts -5 min
saliva_probe_4_ts 5 min
VP007_091123_DigiRelax_Experiment_2023-11-09_14h19.56.411
baseline_instruction_ts 3 min
tsst_prep_ts 5 min
tsst_pres_ts 5 min
tsst_pres_ts 10 min
relaxation_prep_ts 5 min
relaxation_prep_ts 10 min
relaxation_prep_ts 15 min
saliva_probe_4_ts -5 min
saliva_probe_4_ts 5 min
VP015_161123_DigiRelax_Experiment_2023-11-16_09h47.53.124
baseline_instruction_ts 3 min
tsst_prep_ts 5 min
tsst_pres_ts 5 min
tsst_pres_ts 10 min
relaxation_prep_ts 5 min
relaxation_prep_ts 10 min
relaxation_prep_ts 15 min
saliva_probe_4_ts -5 min
saliva_probe_4_ts 5 min
VP011_131123_DigiRelax_Experiment_2023-11-13_14h42.51.249
baseline_instruction_ts 3 min
tsst_prep_ts 5 min
tsst_pres_ts 5 min
tsst_pres_ts 10 min
relaxation_prep_ts 5 min
relaxation_prep_ts 10 min
relaxation_prep_ts 15 min
saliva_probe_4_ts -

/home/muhammad/miniconda3/envs/sql_dep/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/muhammad/miniconda3/envs/sql_dep/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/muhammad/miniconda3/envs/sql_dep/lib/python3.9/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/muhammad/miniconda3/envs/sql_dep/lib/python3.9/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/muhammad/miniconda3/envs/sql_dep/lib/python3.9/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


baseline_instruction_ts 3 min
tsst_prep_ts 5 min
tsst_pres_ts 5 min
tsst_pres_ts 10 min
relaxation_prep_ts 5 min
relaxation_prep_ts 10 min
relaxation_prep_ts 15 min
saliva_probe_4_ts -5 min
saliva_probe_4_ts 5 min
VP022_231123_DigiRelax_Experiment_2023-11-23_09h47.27.026
baseline_instruction_ts 3 min
tsst_prep_ts 5 min
tsst_pres_ts 5 min
tsst_pres_ts 10 min
relaxation_prep_ts 5 min
relaxation_prep_ts 10 min
relaxation_prep_ts 15 min
saliva_probe_4_ts -5 min
saliva_probe_4_ts 5 min
VP005_081123_DigiRelax_Experiment_2023-11-08_14h17.35.226
baseline_instruction_ts 3 min
tsst_prep_ts 5 min
tsst_pres_ts 5 min
tsst_pres_ts 10 min
relaxation_prep_ts 5 min
relaxation_prep_ts 10 min
relaxation_prep_ts 15 min
saliva_probe_4_ts -5 min
saliva_probe_4_ts 5 min
VP008_101123_DigiRelax_Experiment_2023-11-10_10h24.14.323
baseline_instruction_ts 3 min
tsst_prep_ts 5 min
tsst_pres_ts 5 min
tsst_pres_ts 10 min
relaxation_prep_ts 5 min
relaxation_prep_ts 10 min
relaxation_prep_ts 15 min
saliva_probe_4_ts -